# Day 09. Exercise 02
# Metrics

## 0. Imports

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
import joblib

## 1. Preprocessing

1. Create the same dataframe as in the previous exercise.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [300]:
df = pd.read_csv('../data/day-of-week-not-scaled.csv')

In [301]:
X = df.drop('dayofweek', axis=1)
y = df['dayofweek']

In [302]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

## 2. SVM

1. Use the best parameters from the previous exercise and train the model of SVM.
2. You need to calculate `accuracy`, `precision`, `recall`, `ROC AUC`.

 - `precision` and `recall` should be calculated for each class (use `average='weighted'`)
 - `ROC AUC` should be calculated for each class against any other class (all possible pairwise combinations) and then weighted average should be applied for the final metric
 - the code in the cell should display the result as below:

```
accuracy is 0.88757
precision is 0.89267
recall is 0.88757
roc_auc is 0.97878
```

In [303]:
svm = SVC(C=10, class_weight=None, gamma='auto', kernel='rbf', probability=True)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
y_score = svm.predict_proba(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
roc_auc = roc_auc_score(y_test, y_score, multi_class='ovo', average='weighted')


print(f'accuracy is {accuracy:.5f}')
print(f'precision is {precision:.5f}')
print(f'recall is {recall:.5f}')
print(f'roc_auc is {roc_auc:.5f}')

accuracy is 0.88757
precision is 0.89267
recall is 0.88757
roc_auc is 0.97926


## 3. Decision tree

1. The same task for decision tree

In [317]:
dt = DecisionTreeClassifier(class_weight='balanced', criterion= 'gini', max_depth= 22, random_state=21)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
y_pred_proba = dt.predict_proba(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
roc_auc = roc_auc_score(y_test, y_score, multi_class='ovo', average='weighted')

print(f'accuracy is {accuracy:.5f}')
print(f'precision is {precision:.5f}')
print(f'recall is {recall:.5f}')
print(f'roc_auc is {roc_auc:.5f}')

accuracy is 0.89053
precision is 0.89262
recall is 0.89053
roc_auc is 0.97926


## 4. Random forest

1. The same task for random forest.

In [313]:
rfc = RandomForestClassifier(class_weight=None, criterion='gini', max_depth=28, n_estimators=50, random_state=21)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
y_pred_proba = rfc.predict_proba(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
roc_auc = roc_auc_score(y_test, y_score, multi_class='ovo', average='weighted')


print(f'accuracy is {accuracy}')
print(f'precision is {precision}')
print(f'recall is {recall}')
print(f'roc_auc is {roc_auc}')

accuracy is 0.9289940828402367
precision is 0.9300865038851309
recall is 0.9289940828402367
roc_auc is 0.9792598981746689


## 5. Predictions

1. Choose the best model.
2. Analyze: for which `weekday` your model makes the most errors (in % of the total number of samples of that class in your full dataset), for which `labname` and for which `users`.
3. Save the model.

In [ ]:
model = RandomForestClassifier(class_weight=None, criterion='gini', max_depth=28, n_estimators=50, random_state=21)
model.fit(X_train, y_train)
pred = model.predict(X_test)
error_df = X_test.copy()
error_df['predict'] = pred
error_df['error'] = (pred != y_test)
total = X_test.shape[0]

In [307]:
weekday_error = error_df[error_df['error'] == True].groupby('predict').count()['error']/total * 100
weekday_error_max = weekday_error.idxmax()
print(f"weekday with the most errors: {weekday_error_max}")
print(f"{weekday_error[weekday_error_max]:.2f}%")

weekday with the most errors: 6
2.66%


In [308]:
labname_columns = [col for col in error_df.columns if col.startswith('labname_')]
labname_error_counts = error_df[error_df['error'] == True][labname_columns].sum()/total * 100
labname_with_most_errors = labname_error_counts.idxmax()
print(f'Labname with the most errors: {labname_with_most_errors}')
print(f"{labname_error_counts[labname_with_most_errors]:.2f}%")

Labname with the most errors: labname_project1
2.96%


In [309]:
uid_columns = [col for col in error_df.columns if col.startswith('uid_user_')]
uid_error = error_df[error_df['error'] == True][uid_columns].sum()/total * 100
uid_with_most_errors = uid_error.idxmax()
print(f'Labname with the most errors: {uid_with_most_errors}')
print(f"{uid_error[uid_with_most_errors]:.2f}%")

Labname with the most errors: uid_user_19
0.89%


In [ ]:
joblib.dump(model, '../data/best_model_ex02.pkl')

## 6. Function

1. Write a function that takes a list of different models and a corresponding list of parameters (dicts) and returns a dict that contains all the 4 metrics for each model.

In [310]:
def evaluate_models(models, params, X_train, X_test, y_train, y_test):
    metrics = {}
    for model, param in zip(models, params):
        model.set_params(**param)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_score = model.predict_proba(X_test)
        
        metrics[model.__class__.__name__] = {
            'accuracy is': accuracy_score(y_test, y_pred),
            'precision is': precision_score(y_test, y_pred, average='weighted'),
            'recall is': recall_score(y_test, y_pred, average='weighted'),
            'roc_auc is': roc_auc_score(y_test, y_score, multi_class='ovo', average='weighted')
        }
    return metrics

In [312]:
evaluate_models(
    [SVC(), DecisionTreeClassifier(), RandomForestClassifier()],
    [
        {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True},
        {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 39, 'random_state': 21},
        {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 26, 'n_estimators': 50, 'random_state': 21}
    ],
    X_train, X_test, y_train, y_test
)

{'SVC': {'accuracy is': 0.8875739644970414,
  'precision is': 0.8926729169690374,
  'recall is': 0.8875739644970414,
  'roc_auc is': 0.9778575916571358},
 'DecisionTreeClassifier': {'accuracy is': 0.893491124260355,
  'precision is': 0.8961989934143335,
  'recall is': 0.893491124260355,
  'roc_auc is': 0.9355814781907797},
 'RandomForestClassifier': {'accuracy is': 0.9289940828402367,
  'precision is': 0.9309815254490661,
  'recall is': 0.9289940828402367,
  'roc_auc is': 0.9855725850159197}}